<a href="https://colab.research.google.com/github/manishankar23/Transformers_Model/blob/main/Transfomer_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy


In [3]:
class MuliHeadAttention(nn.Module):
  def __init__(self, d_model, num_heads):
    super(MuliHeadAttention, self).__init__()
    assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
    self.d_model=d_model
    self.num_heads=num_heads
    self.d_k=d_model//num_heads

    self.W_q=nn.Linear(d_model, d_model)
    self.W_k=nn.Linear(d_model, d_model)
    self.W_v=nn.Linear(d_model, d_model)
    self.W_o=nn.Linear(d_model, d_model)

  def scaled_dot_product_attention(self,Q, K, V, mask=None):
    attn_scores=torch.matmul(Q, K.transpose(-2, -1))/math.sqrt(self.d_k)
    if mask is not None:
      attn_scores=attn_scores.masked_fill(mask==0, -1e9)

    attn_probs=torch.softmax(attn_scores, dim=-1)
    output=torch.matmul(attn_probs, V)
    return output

  def split_head(self,x):
    batch_size, seq_length, d_model=x.size()
    x=x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1,2)
    return x

  def combine_head(self,x):
    batch_size, _, seq_length, d_k=x.size()
    return x.transpose(1,2).contiguous().view(batch_size, seq_length, self.d_model)

  def forward(self, Q, K, V, mask=None):
    Q=self.split_head(self.W_q(Q))
    K=self.split_head(self.W_k(K))
    V=self.split_head(self.W_v(V))

    attn_output=self.scaled_dot_product_attention(Q, K, V, mask)

    output=self.W_o(self.combine_head(attn_output))
    return output


In [4]:
class PositionWiseFeedForward(nn.Module):
  def __init__(self, d_model, d_ff):
    super(PositionWiseFeedForward, self).__init__()
    self.fc1=nn.Linear(d_model, d_ff)
    self.fc2=nn.Linear(d_ff, d_model)
    self.relu=nn.ReLU()

  def forward(self, x):
    return self.fc2(self.relu(self.fc1(x)))



In [5]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, max_seq_length):
    super(PositionalEncoding, self).__init__()
    pe=torch.zeros(max_seq_length, d_model)
    position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
    div_term=torch.exp(torch.arange(0, d_model, 2).float()*-(math.log(10000.0)/d_model))
    pe[:,0::2]=torch.sin(position*div_term)
    pe[:,1::2]=torch.cos(position*div_term)
    self.register_buffer('pe', pe.unsqueeze(0))

  def forward(self, x):
    return x + self.pe[:,:x.size(1)]

In [6]:

class EncoderLayer(nn.Module):
  def __init__(self, d_model, num_heads, d_ff, dropout):
    super(EncoderLayer, self).__init__()
    self.self_attn=MuliHeadAttention(d_model, num_heads)
    self.feed_forward=PositionWiseFeedForward(d_model, d_ff)
    self.norm1=nn.LayerNorm(d_model)
    self.norm2=nn.LayerNorm(d_model)
    self.dropout=nn.Dropout(dropout)

  def forward (self, x, mask):
    attn_output=self.self_attn(x, x, x, mask)
    x=self.norm1(x+self.dropout(attn_output))
    ff_output=self.feed_forward(x)
    x=self.norm2(x+self.dropout(ff_output))
    return x

In [7]:
class DecoderLayer(nn.Module):
  def __init__(self, d_model, num_heads, d_ff, dropout):
    super(DecoderLayer, self).__init__()

    self.self_attn=MuliHeadAttention(d_model, num_heads)
    self.cross_attn=MuliHeadAttention(d_model, num_heads)
    self.feed_forward=PositionWiseFeedForward(d_model, d_ff)
    self.norm1=nn.LayerNorm(d_model)
    self.norm2=nn.LayerNorm(d_model)
    self.norm3=nn.LayerNorm(d_model)
    self.dropout=nn.Dropout(dropout)

  def forward(self, x, enc_output, src_mask, tgt_mask):
    attn_output=self.self_attn(x, x, x, tgt_mask)
    x=self.norm1(x+self.dropout(attn_output))
    attn_output=self.cross_attn(x, enc_output, enc_output, src_mask)
    x=self.norm2(x+self.dropout(attn_output))
    ff_output=self.feed_forward(x)
    x=self.norm3(x+self.dropout(ff_output))
    return x

In [8]:
class Transformer(nn.Module):
  def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
    super(Transformer, self).__init__()

    self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
    self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)

    self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
    self.encoder_layers=nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
    self.decoder_layers=nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

    self.fc=nn.Linear(d_model, tgt_vocab_size)
    self.droupout=nn.Dropout(dropout)

  def generate_mask(self,src,tgt):
    src_mask=(src!=0).unsqueeze(1).unsqueeze(2)
    tgt_mask=(tgt!=0).unsqueeze(1).unsqueeze(3)
    seq_length=tgt.size(-1)
    nopeak_mask=(1 - torch.triu(torch.ones(seq_length, seq_length),diagonal=1)).bool()
    tgt_mask=tgt_mask & nopeak_mask.unsqueeze(0).unsqueeze(0)
    return src_mask, tgt_mask

  def forward(self, src, tgt):
    src_mask, tgt_mask=self.generate_mask(src,tgt)
    src_embedded=self.droupout(self.positional_encoding(self.encoder_embedding(src)))
    tgt_embedded=self.droupout(self.positional_encoding(self.decoder_embedding(tgt)))
    enc_output=src_embedded
    for enc_layer in self.encoder_layers:
      enc_output=enc_layer(enc_output, src_mask)
    dec_output = tgt_embedded  # Initialize dec_output
    for dec_layer in self.decoder_layers:
      dec_output=dec_layer(dec_output, enc_output, src_mask, tgt_mask)
    output=self.fc(dec_output)
    return output


In [9]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

In [10]:

src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(20):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

    transformer.eval()

# Generate random sample validation data
val_src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
val_tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

with torch.no_grad():

    val_output = transformer(val_src_data, val_tgt_data[:, :-1])
    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), val_tgt_data[:, 1:].contiguous().view(-1))
    print(f"Validation Loss: {val_loss.item()}")


Epoch: 1, Loss: 8.690166473388672
Epoch: 2, Loss: 8.545184135437012
Epoch: 3, Loss: 8.465479850769043
Epoch: 4, Loss: 8.39908504486084
Epoch: 5, Loss: 8.321629524230957
Epoch: 6, Loss: 8.221704483032227
Epoch: 7, Loss: 8.114349365234375
Epoch: 8, Loss: 8.01082706451416
Epoch: 9, Loss: 7.9050164222717285
Epoch: 10, Loss: 7.8018107414245605
Epoch: 11, Loss: 7.701848983764648
Epoch: 12, Loss: 7.599315166473389
Epoch: 13, Loss: 7.491184234619141
Epoch: 14, Loss: 7.3853044509887695
Epoch: 15, Loss: 7.279462814331055
Epoch: 16, Loss: 7.179128170013428
Epoch: 17, Loss: 7.0812907218933105
Epoch: 18, Loss: 6.983625888824463
Epoch: 19, Loss: 6.890336036682129
Epoch: 20, Loss: 6.796113014221191
Validation Loss: 8.688786506652832
